In [2]:
!pip install feedparser

  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=cf74ec1d71c36e58c0568bcfab8a9eacd2038a7ed982079de5518bae30a96037
  Stored in directory: c:\users\laura\appdata\local\pip\cache\wheels\65\7a\a7\78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [3]:
# Imports
import pandas as pd
import feedparser
import plotly.express as px
from datetime import datetime, timedelta
import ipywidgets as widgets
from ipywidgets import interact

In [4]:
# Data file import
data = feedparser.parse("http://www.ign.es/ign/RssTools/sismologia.xml")

# Exercise 3 (4pts)
a) Create a map with the earthquakes that have occurred in the last ten days from the IGN information. (2 pts)
http://www.ign.es/ign/RssTools/sismologia.xml
*   The date, magnitude, longitude and latitude will be displayed at each point when you mouse over it.
*   The color and size of the dot have to depend on the magnitude.

In [6]:
# We prepare the data in a dataframe
earthquakes = []
for item in data.entries:
    earthquakes.append((item.title[17:], float(item.description[41:44]), float(item.geo_lat), float(item.geo_long)))
df = pd.DataFrame(earthquakes, columns=['Date', 'Magnitude', 'Latitude', 'Longitude'])

# We create the graph from the dataframe
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", zoom=8, height=700, width=1500, hover_data= ["Date", "Magnitude"], color='Magnitude', size='Magnitude')
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, 
    margin={"r":0,"t":0,"l":0,"b":0})

# We show the graph
fig.show()

b) Using ipywidgets (or Dash), create a map like above but with interactive controls for: (2 pts)
*   Show only earthquakes in a certain magnitude range (FloatRangeSlider)
*   Change the number of days (list with values ​​from 1 to 10, Dropdown element)

In [8]:
# Añadimos el slider de magnitud y el selector de días
@interact(Magnitude = widgets.FloatRangeSlider(
            value=[df['Magnitude'].min(), df['Magnitude'].max()],
            min=df['Magnitude'].min(),
            max=df['Magnitude'].max(),
            step=0.1,
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'),
          Days = widgets.Dropdown(
            options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            value=10,
            disabled=False,
        )
)

# We create the function
def cargarMapa(Magnitude, Days):
    terremotos = []
    for item in data.entries:
        # If the magnitude is between the minimum and maximum chosen by the slider
        # and if the date is after today's day subtracted from the value chosen in the dropdown,
        # I add the result to the array earthquakes
        if (float(item.description[41:44]) >= Magnitude[0] and float(item.description[41:44]) <= Magnitude[1]) and (pd.to_datetime(item.title[17:]) >= datetime.today() - timedelta(Days)):
            terremotos.append((item.title[17:], float(item.description[41:44]), float(item.geo_lat), float(item.geo_long)))
    df = pd.DataFrame(terremotos, columns=['Date', 'Magnitude', 'Latitude', 'Longitude'])

    # We create the graph from the generated dataframe
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", zoom=8, height=700, width=1500, hover_data= ["Date", "Magnitude"], color='Magnitude', size='Magnitude')
    fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, 
        margin={"r":0,"t":0,"l":0,"b":0})

    # We show the graph
    fig.show()

interactive(children=(FloatRangeSlider(value=(2.6, 3.8), continuous_update=False, description='Magnitude', max…